# evaluating effects of missing data threshold and population assignment on PCA, t-SNE, and dxy 

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE

In [2]:
# the path to your HDF5 formatted snps file
data = "../plate1/H_st_11rm_outfiles/H_st_11rm.snps.hdf5"

Let's define populations Western Inambari (Inam), Purus-Madeira (Puru), Jiparana-Guapore (JiGu), Jiparaná-Roosevelt (Mach), Roosevelt-Aripuana (Roar), Aripuana-Sucunduri (ArSu), Sucunduri-Tapajos (SuTa), and Western Para (Para). 

These assignments are made only to color points in the PCA analysis below.

In [3]:
imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

# minimum % of samples that must be present in each SNP from each population: do 5 subsets to see robustness of impu
#Because there is a lot of missing data, we want to make sure that results are not biased due to imputing too many haplotypes
#sampling across different minmaps will help address this. If the data are more organized by population with more missing data, then this could be bias in the imputation from small sample sizes
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

let's run PCA for different values of minmap to see how varying missing data affects our results

note we are assuming K=7 (western inambari not sampled) which is likely high, but k-means clustering will cluster samples independent of a priori geographic assignment

Here we are using k-means clustering to assign individuals to populations independently of our a priori geographic assignments in order to circumscribe populations from which to impute missing haplotypes.

The PCA plots can then be colored based on our geographic assignments above

In [4]:
#init pca object with input data and (optional) parameter options

pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method=7,
)
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method=7,
)
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method=7,
)
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method=7,
)
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method=7,
)

Kmeans clustering: iter=0, K=7, mincov=0.9, minmap={'global': 0.85}
Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1570811
Filtered (minmap): 1477593
Filtered (combined): 1575512
Sites after filtering: 127149
Sites containing missing values: 124182 (97.67%)
Missing values in SNP matrix: 507776 (6.66%)
Imputation: 'sampled'; (0, 1, 2) = 59.6%, 4.9%, 35.5%
{0: ['H_oc_T355_jigu', 'H_ro_A547_ma', 'H_ro_J213_ma', 'H_ro_J305_ma', 'H_ro_J508_ma', 'H_ro_J774_ma', 'H_ro_J775_ma', 'H_ro_J796_ma', 'H_ro_T1842_ma', 'H_ro_T366_ma', 'H_ro_T385_ma', 'H_ro_T471_ma'], 1: ['H_ro_A409_ma', 'H_st_77860_arsu', 'H_st_80727_arsu', 'H_st_81143_arsu', 'H_st_A14546_suta', 'H_st_A16571_pa', 'H_st_A272_arsu', 'H_st_T10207_suta', 'H_st_T17858_pa'], 2: ['H_oc_A311_jigu', 'H_oc_T15847_jigu'], 3: ['H_st_A11597_pa', 'H_st_A15208_pa', 'H_st_A15210_pa', 'H_st_A7597_pa', 'H_st_T16744_pa'], 4: ['H_pe_A24346_pu', 'H_ro_A410_ma', 'H_ro_A521_ma', 'H_st_80800_ars

In [5]:
# # run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 1337/11966
Subsampling SNPs: 1540/14023
Subsampling SNPs: 1638/14811
Subsampling SNPs: 1563/14267
Subsampling SNPs: 346/2967


In [6]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2ad04e8d7450>,
 <toyplot.coordinates.Cartesian at 0x2ad04e8d73d0>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t097bfbe525e64b4eb65e7909b72406e3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -3 0 3 6 PC0 (16.0%) explained -4 0 4 8 PC1 (8.6%) explained Puru JiGu Mach Roar ArSu SuTa Para

In [7]:
pca2.draw()

(<toyplot.canvas.Canvas at 0x2ad04dad5190>,
 <toyplot.coordinates.Cartesian at 0x2ad04dade050>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t453e2c5596cb4fbbbd3f0ba158750433" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4 0 4 8 PC0 (16.0%) explained -2 0 2 5 8 PC1 (6.9%) explained Puru JiGu Mach Roar ArSu SuTa Para

In [8]:
pca3.draw()

(<toyplot.canvas.Canvas at 0x2ad04dac2350>,
 <toyplot.coordinates.Cartesian at 0x2ad04dac2790>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t12a1fad4a8ab4c62ba825d7ca209cb83" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4 0 4 8 PC0 (17.4%) explained -3 0 3 6 PC1 (6.9%) explained Puru JiGu Mach Roar ArSu SuTa Para

In [9]:
pca4.draw()

(<toyplot.canvas.Canvas at 0x2ad04d73e110>,
 <toyplot.coordinates.Cartesian at 0x2ad04d73e150>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8ab1afb4ba564f7ca203297109604301" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4 0 4 8 PC0 (19.8%) explained -2 0 2 4 6 PC1 (6.9%) explained Puru JiGu Mach Roar ArSu SuTa Para

In [10]:
pca5.draw()

(<toyplot.canvas.Canvas at 0x2ad04dae7ad0>,
 <toyplot.coordinates.Cartesian at 0x2ad04daf5090>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t637ecd86481147f9bcbc9aa7b77c32ab" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -2 0 2 PC0 (11.8%) explained -2 -1 0 1 2 3 PC1 (7.4%) explained Puru JiGu Mach Roar ArSu SuTa Para

As you can see, for varying degrees of missing data, we generally get the same or similar results except for very low values of missing data (minmap=0.95) probably because there isnt enough data to see the patterns.

there are three to five clusters of points here that correspond pretty clearly to many river barriers and these are consistent among runs.

Now we can write the PCA results to a file

In [11]:
# # store the PC axes as a dataframe
df4 = pd.DataFrame(pca4.pcaxes[0], index=pca4.names)

# # write the PC axes to a CSV file
df4.to_csv("H_st_pca_85minmap_12Jan2022.csv")

# # show the first ten samples and the first 10 PC axes
df4.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
H_oc_A311_jigu,0.18,5.14,9.73,-3.24,-0.41,-0.05,0.32,-0.19,-0.14,-0.49
H_oc_T15847_jigu,-0.36,5.34,9.72,-3.23,-0.38,-0.10,0.17,-0.52,0.01,-0.01
H_oc_T355_jigu,7.40,-1.67,0.14,-0.17,-0.01,-0.90,1.17,0.03,0.19,-0.36
H_pe_A24346_pu,0.25,5.16,3.16,13.01,-0.71,1.56,0.90,0.65,0.24,0.46
H_ro_A409_ma,-2.20,-1.21,-0.26,-0.95,-0.77,0.76,2.45,-0.66,1.74,1.22
H_ro_A410_ma,-2.92,-0.29,0.04,0.10,-0.71,-0.06,-1.19,-1.59,0.03,-1.20
H_ro_A521_ma,-1.91,-1.19,0.81,-0.17,-1.07,-0.90,-0.97,0.12,3.08,1.90
H_ro_A547_ma,8.17,-2.33,0.33,0.20,0.67,0.11,1.64,-0.67,0.02,1.06
H_ro_A551_ma,-2.41,0.02,-0.26,-0.38,-1.81,0.10,-0.33,2.24,1.08,-0.69
H_ro_J213_ma,8.13,-1.61,0.24,-0.23,0.42,0.05,0.87,-0.31,-0.01,1.14


Now let's look at if and how t-SNE differs from PCA

In [14]:
pca.run_tsne(subsample=True, perplexity=6.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=6.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=6.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=6.0, n_iter=10000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=6.0, n_iter=1000000, seed=123)

Subsampling SNPs: 1337/11966
Subsampling SNPs: 1540/14023
Subsampling SNPs: 1638/14811
Subsampling SNPs: 1563/14267
Subsampling SNPs: 346/2967


In [15]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc2b2ad2b12ab4d65b1d0c97efa1d7407" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4000 -2000 0 2000 4000 TSNE component 1 -4000 -2000 0 2000 4000 TSNE component 2 Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t48ddc562b42247aca5f2caec1ff97300" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -200 0 200 400 TSNE component 1 -100 0 100 200 TSNE component 2 Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t5c4e40d2f6114fe3b182ebe7b6ac1fe1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -2000 -1000 0 1000 TSNE component 1 -200 -100 0 100 200 TSNE component 2 Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="td41efe2e6c3e4fb187057216a1353fc6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -500 0 500 1000 1500 TSNE component 1 -2000 -1000 0 TSNE component 2 Puru JiGu Mach Roar ArSu SuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t31c089f9e6d24bcdb0a547eacc15f2b3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -1000 0 1000 2000 TSNE component 1 -600 -300 0 300 TSNE component 2 Puru JiGu Mach Roar ArSu SuTa Para

amazingly with t-sne we see significantly more clusters

note that these results are sensitive to values of perplexity and starting seed, so you can play with those parameters to get more interpretable results

Then we can again write the t-sne results to a file

In [16]:
# # store the PC axes as a dataframe
df4 = pd.DataFrame(pca4.pcaxes[0], index=pca4.names)

# # write the PC axes to a CSV file
df4.to_csv("H_st_TSNE_85minmap_12Jan2022.csv")

# # show the first ten samples and the first 10 PC axes
df4.iloc[:10, :10].round(2)

,0,1
H_oc_A311_jigu,-138.78,-2312.57
H_oc_T15847_jigu,1324.32,-470.17
H_oc_T355_jigu,87.74,-503.69
H_pe_A24346_pu,-28.70,389.99
H_ro_A409_ma,0.65,260.58
H_ro_A410_ma,177.84,304.17
H_ro_A521_ma,58.25,220.59
H_ro_A547_ma,70.57,-589.86
H_ro_A551_ma,-107.97,-45.44
H_ro_J213_ma,56.00,-543.80


Let's skip this for now, but the next section of code does 10,000 TSNE replicates for downstream analysis using randomply generated values for starting seed and "perplexity"

In [17]:
# !rm -r H_st_TSNE
# !mkdir H_st_TSNE
# import random
# for i in range(10000):
#     pca3.run_tsne(subsample=True, perplexity=random.randrange(3,8), n_iter=100000, seed=random.randrange(100,9999))
#     df4 = pd.DataFrame(pca3.pcaxes[0], index=pca4.names)
#     df4.to_csv("./H_st_TSNE/H_st_TSNE_rep"+str(i)+".csv")

#### Now let's look at just the western clade, and here we assume K=4; again clustering is independent of geography

In [28]:
#RUN AGAIN WITH WESTERN POPULATIONS ONLY

imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
#"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
#"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
#"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [29]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.75,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.75,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.75,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.75,
    impute_method=4,
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.75,
    impute_method=4,
)

Kmeans clustering: iter=0, K=4, mincov=0.9, minmap={'global': 0.75}
Samples: 39
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 32470
Filtered (mincov): 1586520
Filtered (minmap): 1307366
Filtered (combined): 1589350
Sites after filtering: 113311
Sites containing missing values: 103733 (91.55%)
Missing values in SNP matrix: 229410 (5.19%)
Imputation: 'sampled'; (0, 1, 2) = 59.2%, 4.5%, 36.3%
{0: ['H_ro_A409_ma', 'H_st_77860_arsu', 'H_st_80727_arsu', 'H_st_81143_arsu', 'H_st_A14546_suta', 'H_st_A272_arsu', 'H_st_T10207_suta'], 1: ['H_pe_A24346_pu', 'H_ro_A521_ma', 'H_ro_A551_ma', 'H_st_78249_arsu', 'H_st_80800_arsu', 'H_st_81279_suta', 'H_st_85680_arsu', 'H_st_85970_arsu', 'H_st_A273_arsu', 'H_st_J525_arsu', 'H_st_J530_arsu', 'H_st_J536_arsu', 'H_st_J572_arsu'], 2: ['H_st_J364_roar', 'H_st_J368_roar', 'H_st_J370_roar', 'H_st_J374_roar', 'H_st_J408_roar', 'H_st_J621_roar', 'H_st_J664_roar', 'H_st_J665_roar', 'H_st_J711_roar', 'H_st_J762_roar', 'H_st_J765_roar'],

In [30]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 12657/143078
Subsampling SNPs: 12712/143877
Subsampling SNPs: 12666/143209
Subsampling SNPs: 12556/141633
Subsampling SNPs: 1052/9578


In [31]:
pca.draw()
pca2.draw()
pca3.draw()
pca4.draw()
pca5.draw()

(<toyplot.canvas.Canvas at 0x2ad05940be10>,
 <toyplot.coordinates.Cartesian at 0x2ad05940b710>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7836089b667248a48ddf6d516b49c6d3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -10 0 10 PC0 (7.3%) explained 0 10 20 30 40 PC1 (7.0%) explained Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3cafdce580a14a3f9b8ecf3d20b8d84f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -10 0 10 20 30 PC0 (7.4%) explained -10 0 10 20 30 PC1 (6.9%) explained Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t02edb23bea5b40d989efb5f419095f57" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -10 0 10 20 30 PC0 (7.4%) explained 0 10 20 30 PC1 (7.2%) explained Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tdfe2006e06134e16847ecad20b65bed8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -5 0 5 10 PC0 (7.5%) explained 0 10 20 30 40 PC1 (7.2%) explained Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t31dc393b37fe48578022b26fe89f737e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0 5 10 PC0 (9.5%) explained -2 0 2 4 PC1 (7.9%) explained Puru Roar ArSu SuTa

When we zoom in on one clade, we can see that there is even more structure than we initially thought

In [32]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)

Subsampling SNPs: 12657/143078
Subsampling SNPs: 12712/143877
Subsampling SNPs: 12666/143209
Subsampling SNPs: 12556/141633
Subsampling SNPs: 1052/9578


In [33]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te101adfcc66543f3820ba8292c4b9237" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0 1000 2000 TSNE component 1 -500 0 500 1000 TSNE component 2 Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="taa7db6b80ae049fc9e9dd0a660502439" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -1000 -500 0 500 TSNE component 1 -250 0 250 500 750 TSNE component 2 Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t85df4eb93f974b4ca05572919762ce23" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -300 0 300 600 900 TSNE component 1 -500 0 500 1000 TSNE component 2 Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te19072f1f8904900bf15c2cc3bfd5248" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0 500 1000 TSNE component 1 -100 0 100 TSNE component 2 Puru Roar ArSu SuTa

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3ccf31fc336045f5941313edb1e32fb4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -300 -200 -100 0 100 200 TSNE component 1 -250 0 250 TSNE component 2 Puru Roar ArSu SuTa

Now let's look at just the eastern clade, and here we assume K=3; again clustering is independent of geography

In [34]:
#RUN AGAIN WITH "EASTERN" POPULATIONS ONLY

imap = {
#"ref": ["reference"],
#"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
#"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
#"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
#"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}


# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [35]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.75,
    impute_method=3,
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.75,
    impute_method=3,
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.75,
    impute_method=3,
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.75,
    impute_method=3,
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.75,
    impute_method=3,
)

Kmeans clustering: iter=0, K=3, mincov=0.9, minmap={'global': 0.75}
Samples: 21
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 18332
Filtered (mincov): 1549286
Filtered (minmap): 1327506
Filtered (combined): 1551658
Sites after filtering: 151003
Sites containing missing values: 132272 (87.60%)
Missing values in SNP matrix: 207320 (6.54%)
Imputation: 'sampled'; (0, 1, 2) = 58.9%, 4.4%, 36.7%
{0: ['H_oc_T355_jigu', 'H_ro_A547_ma', 'H_ro_J213_ma', 'H_ro_J305_ma', 'H_ro_J508_ma', 'H_ro_J774_ma', 'H_ro_J775_ma', 'H_ro_J796_ma', 'H_ro_T1842_ma', 'H_ro_T366_ma', 'H_ro_T385_ma', 'H_ro_T471_ma'], 1: ['H_st_A11597_pa', 'H_st_A15208_pa', 'H_st_A15210_pa', 'H_st_A16571_pa', 'H_st_A7597_pa', 'H_st_T16744_pa', 'H_st_T17858_pa'], 2: ['H_oc_A311_jigu', 'H_oc_T15847_jigu']}

Kmeans clustering: iter=1, K=3, mincov=0.8625, minmap={0: 0.75, 1: 0.75, 2: 0.75}
Samples: 21
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 18332
Filtered (mincov): 1549286
Fil

In [36]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 2038/18731


In [37]:
pca.draw()
pca2.draw()
pca3.draw()
pca4.draw()
pca5.draw()

(<toyplot.canvas.Canvas at 0x2ad05944d5d0>,
 <toyplot.coordinates.Cartesian at 0x2ad05944dc50>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc6ae260bb3f04e97b3bbe40d65690163" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -10 0 10 20 PC0 (30.6%) explained -10 0 10 20 30 PC1 (15.7%) explained JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t297a60318ff14ccab96993064d804e7c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -10 0 10 20 PC0 (29.4%) explained -10 0 10 20 30 PC1 (15.8%) explained JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tcb7f6062a50e4fc4a6ac810bd4a449e1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -10 0 10 20 PC0 (31.2%) explained -10 0 10 20 30 PC1 (15.9%) explained JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t19167eda0a644e34adc932d7ca24971a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -10 0 10 20 PC0 (30.7%) explained -10 0 10 20 30 PC1 (16.2%) explained JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc0ad5a50d6e54aceb8d90f99958be653" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -5 0 5 10 PC0 (34.2%) explained -5 0 5 10 15 PC1 (16.2%) explained JiGu Mach Para

In [38]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca2.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca4.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=123)

Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 11240/118264
Subsampling SNPs: 2038/18731


In [39]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t61bdc5b9182a49eeb31986840af8733e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -500 0 500 TSNE component 1 -200 -100 0 100 TSNE component 2 JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te2fba18ac3314eb9864c70bb55cdf1ef" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -500 0 500 TSNE component 1 -250 0 250 TSNE component 2 JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t945c94cd29c946a180827b4deb822715" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -100 0 100 TSNE component 1 -500 0 500 TSNE component 2 JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t4d14ed026710458db61be4d60bde700e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -400 -200 0 200 400 TSNE component 1 -1000 -500 0 500 TSNE component 2 JiGu Mach Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8394b61a16dd4c70ba571c7a0455f357" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -100 0 100 200 300 400 TSNE component 1 -500 -250 0 250 TSNE component 2 JiGu Mach Para

Now let's see if the results hold up to a different clustering algorithm so I will group samples into "populations" based on results from the structure analysis to see if PCA and t-SNE look the same. Here we use "sample" to impute from a priori defined populations

In [48]:
imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T15847_jigu"],
"Mach": [ "H_oc_T355_jigu", "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"RoArSuTa": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar","H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu","H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta","H_st_A16571_pa"],
"Para": ["H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}
# minimum % of samples that must be present in each SNP from each group
minmap1 = {i: 0.55 for i in imap}
minmap2 = {i: 0.65 for i in imap}
minmap3 = {i: 0.75 for i in imap}
minmap4 = {i: 0.85 for i in imap}
minmap5 = {i: 0.95 for i in imap}

In [49]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap1,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca2 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap2,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap3,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca4 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
)
# init pca object with input data and (optional) parameter options
pca5 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap5,
    mincov=0.85,
    impute_method="sample",
)

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1477593
Filtered (minmap): 1573111
Filtered (combined): 1600330
Sites after filtering: 102331
Sites containing missing values: 99364 (97.10%)
Missing values in SNP matrix: 460492 (7.50%)
Imputation: 'sampled'; (0, 1, 2) = 59.9%, 4.2%, 35.9%
Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1477593
Filtered (minmap): 1577389
Filtered (combined): 1600629
Sites after filtering: 102032
Sites containing missing values: 99065 (97.09%)
Missing values in SNP matrix: 458127 (7.48%)
Imputation: 'sampled'; (0, 1, 2) = 59.9%, 4.3%, 35.8%
Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1477593
Filtered (minmap): 1625002
Filtered (combined): 1630681
Sites after filtering: 71980
Sites containing missing values: 69013 (95.88%)
Missing values in SNP matrix: 272824 (6.32%)

In [50]:
# run the PCA analysis
pca.run()
pca2.run()
pca3.run()
pca4.run()
pca5.run()

Subsampling SNPs: 9499/102331
Subsampling SNPs: 9468/102032
Subsampling SNPs: 6992/71980
Subsampling SNPs: 1831/16817
Subsampling SNPs: 786/6821


In [51]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t74e6b6c46e564ac19cb85439ce3e25ba" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -10 0 10 20 PC0 (18.0%) explained -10 0 10 20 PC1 (8.6%) explained Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tac9171f576564e1388c272cd7e62232e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -10 0 10 20 PC0 (17.4%) explained -10 0 10 20 PC1 (8.9%) explained Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t20a540bd55ba4ac7b672d3864f43321e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -10 0 10 20 PC0 (18.3%) explained -5 0 5 10 15 20 PC1 (8.2%) explained Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc5c8359cfdd24f14b81e86369ac73bdc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4 0 4 8 PC0 (17.6%) explained -4 0 4 8 PC1 (8.0%) explained Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t2f6adf13b18540208b7637d787ab1174" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -2 0 2 5 PC0 (16.6%) explained -2 0 2 4 6 PC1 (9.1%) explained Puru JiGu Mach RoArSuTa Para

In [52]:
pca.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)
pca2.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=123)
pca3.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)
pca4.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=123)
pca5.run_tsne(subsample=True, perplexity=4.0, n_iter=1000000, seed=223)

Subsampling SNPs: 9499/102331
Subsampling SNPs: 9468/102032
Subsampling SNPs: 6992/71980
Subsampling SNPs: 1831/16817
Subsampling SNPs: 786/6821


In [53]:
pca.draw();
pca2.draw();
pca3.draw();
pca4.draw();
pca5.draw();

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8202b068916c4e58b2355498ded38768" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -400 -200 0 200 400 TSNE component 1 -100 0 100 200 300 400 TSNE component 2 Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tdec354502d57406cbb8814450dda489e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -4000 -2000 0 TSNE component 1 -7500 -5000 -2500 0 TSNE component 2 Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t0c6fb68674f647cc8d9bcfe3d1c923c6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -500 0 500 1000 TSNE component 1 0 500 1000 TSNE component 2 Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb77696d34fe54e7db40e884454aa49d2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -50 0 50 100 TSNE component 1 -50 0 50 100 150 TSNE component 2 Puru JiGu Mach RoArSuTa Para

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t742de2111c094b81907afd448e0b4a1b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -200 -100 0 100 TSNE component 1 -50 0 50 TSNE component 2 Puru JiGu Mach RoArSuTa Para

Amazingly, the results look rather similar to the initial results of imputation by popuilations defined by the k-means clustering analysis

(You can place your cursor on a point to see the label)

# Genetic Distances (dxy)

Now let's generate distance matrices for EEMs and see if imputation effects measures of distance. First look at imputation results based on structure assignments, and then look at imputation results based on a priori assignments

In [54]:
imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T15847_jigu"],
"Mach": [ "H_oc_T355_jigu", "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"RoArSuTa": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar","H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu","H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta","H_st_A16571_pa"],
"Para": ["H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

minmap4 = {i: 0.85 for i in imap}


In [55]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1477593
Filtered (minmap): 1685180
Filtered (combined): 1685844
Sites after filtering: 16817
Sites containing missing values: 13850 (82.36%)
Missing values in SNP matrix: 33448 (3.31%)
Imputation: 'sampled'; (0, 1, 2) = 60.9%, 4.7%, 34.4%


In [56]:
# save to a CSV file
dist.dists.to_csv("M_ru_distances_12Jan2022.csv")

# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "M_ru_distances_eems_12Jan2022.csv",
    header=None,
    index=False,
    sep=" ",
)

In [60]:
imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T15847_jigu","H_oc_T355_jigu"],
"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

minmap4 = {i: 0.85 for i in imap}

In [61]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist2 = Distance(
    data=data, 
    imap=imap,
    minmap=minmap4,
    mincov=0.85,
    impute_method="sample",
    subsample_snps=False,
)
dist2.run()

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1477593
Filtered (minmap): 1661930
Filtered (combined): 1663377
Sites after filtering: 39284
Sites containing missing values: 36317 (92.45%)
Missing values in SNP matrix: 91363 (3.88%)
Imputation: 'sampled'; (0, 1, 2) = 62.2%, 4.0%, 33.8%


In [62]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

# get list of concatenated names from each group
ordered_names = []
for group in dist2.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist2.dists[ordered_names].T[ordered_names]

toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="tec913a64e7f64537bd06e1e0d1b0fc50" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.204733 0.201701 0.208301 0.212820 0.209253 0.207171 0.206458 0.206993 0.206220 0.208242 0.201344 0.206577 0.206339 0.209490 0.196052 0.188440 0.192484 0.190521 0.190819 0.197003 0.189392 0.191592 0.186775 0.189867 0.191711 0.190938 0.189332 0.192127 0.186121 0.190284 0.182316 0.190759 0.188321 0.188262 0.186478 0.190284 0.188738 0.190521 0.183445 0.187608 0.190105 0.188916 0.188024 0.185229 0.188619 0.193138 0.191116 0.187251 0.183267 0.187667 0.184159 0.184991 0.196706 0.203187 0.208420 0.199144 0.202355 0.200392 0.195933 H_pe_A24346_pu 1 0.204733 0.000000 0.047036 0.162930 0.169352 0.165785 0.162395 0.168460 0.163168 0.163109 0.167509 0.161206 0.162990 0.161919 0.163763 0.158054 0.160552 0.163763 0.157043 0.160076 0.160790 0.155557 0.155497 0.151394 0.161979 0.157519 0.160076 0.157043 0.160790 0.153714 0.160968 0.148719 0.159065 0.154130 0.154308 0.158946 0.159898 0.158471 0.158471 0.153297 0.159006 0.157341 0.154962 0.157995 0.153535 0.155497 0.159422 0.155735 0.162693 0.154308 0.157757 0.153059 0.154368 0.165249 0.164714 0.174347 0.160195 0.165904 0.167271 0.162930 H_oc_A311_jigu 2 0.201701 0.047036 0.000000 0.162871 0.169293 0.166320 0.163168 0.166736 0.164655 0.162930 0.166974 0.162574 0.164714 0.164001 0.164893 0.157281 0.160492 0.160730 0.156746 0.159065 0.159541 0.153238 0.154724 0.149670 0.160611 0.156033 0.159660 0.159600 0.159541 0.151038 0.160671 0.147351 0.157935 0.153238 0.153654 0.158173 0.159125 0.156746 0.155914 0.151930 0.157757 0.153357 0.152286 0.155081 0.152405 0.153178 0.157222 0.155676 0.160136 0.151989 0.156984 0.150502 0.152881 0.164239 0.164774 0.175001 0.159065 0.164774 0.165666 0.161325 H_oc_T15847_jigu 3 0.208301 0.162930 0.162871 0.000000 0.052566 0.055182 0.055361 0.057026 0.054588 0.058453 0.054290 0.058928 0.054528 0.048582 0.056253 0.138788 0.138312 0.134269 0.140156 0.138312 0.140572 0.133199 0.126717 0.137004 0.139502 0.138015 0.129988 0.136826 0.137599 0.128977 0.131117 0.133258 0.138134 0.131177 0.129096 0.133734 0.133258 0.136945 0.131474 0.132366 0.136766 0.137718 0.134031 0.131474 0.135220 0.139085 0.139323 0.134209 0.136529 0.133377 0.131474 0.136053 0.135934 0.139918 0.135458 0.138193 0.134269 0.133199 0.131474 0.130701 H_oc_T355_jigu 4 0.212820 0.169352 0.169293 0.052566 0.000000 0.054944 0.052982 0.058810 0.056847 0.061069 0.054171 0.060831 0.060593 0.051198 0.062913 0.142237 0.141999 0.140810 0.144794 0.142832 0.146043 0.139145 0.135161 0.141523 0.143664 0.144675 0.136529 0.142059 0.147232 0.135161 0.137539 0.137420 0.145745 0.139977 0.134804 0.139799 0.137777 0.141702 0.137896 0.140453 0.140810 0.144259 0.138669 0.135874 0.141286 0.143724 0.145745 0.140750 0.144259 0.141821 0.136469 0.141405 0.141167 0.146935 0.139264 0.142951 0.136766 0.135220 0.133853 0.134388 H_ro_A547_ma 5 0.209253 0.165785 0.166320 0.055182 0.054944 0.000000 0.055599 0.057145 0.057918 0.057263 0.053220 0.058453 0.058928 0.053696 0.058869 0.139264 0.140929 0.136410 0.140275 0.138669 0.141642 0.134269 0.130998 0.139026 0.138669 0.139918 0.132961 0.138372 0.139977 0.131355 0.135161 0.136707 0.139204 0.132961 0.131355 0.136945 0.135755 0.138015 0.134447 0.134864 0.139145 0.141167 0.135101 0.133020 0.137480 0.141464 0.141940 0.136112 0.139740 0.138491 0.133972 0.139145 0.138788 0.143010 0.134626 0.137599 0.131890 0.131296 0.127906 0.129869 H_ro_J213_ma 6 0.207171 0.162395 0.163168 0.055361 0.052982 0.055599 0.000000 0.052090 0.051674 0.051139 0.052447 0.053398 0.057680 0.

<svg class="toyplot-canvas-Canvas" height="600px" id="tddb39c9d39dd49adb27f11e23e1dc686" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.191045 0.189568 0.190968 0.193158 0.190485 0.189772 0.187990 0.189339 0.188754 0.194354 0.185266 0.189466 0.191376 0.187735 0.178011 0.176280 0.180201 0.178495 0.176433 0.183255 0.177044 0.182288 0.180761 0.178037 0.179157 0.179539 0.176077 0.175873 0.172284 0.178902 0.176713 0.176764 0.173989 0.173149 0.177350 0.180022 0.175771 0.179106 0.173913 0.177960 0.173989 0.177120 0.177553 0.172818 0.173302 0.177935 0.177350 0.179971 0.174397 0.179768 0.174702 0.176230 0.184223 0.188041 0.195347 0.184961 0.186615 0.185674 0.184019 H_pe_A24346_pu 1 0.191045 0.000000 0.051115 0.164214 0.166862 0.163069 0.163273 0.162916 0.163349 0.161134 0.167447 0.161567 0.162305 0.163960 0.160472 0.152479 0.153498 0.157418 0.155458 0.153090 0.160218 0.153090 0.156960 0.156247 0.154847 0.157036 0.159607 0.155534 0.154567 0.150927 0.159276 0.154032 0.156629 0.152530 0.152403 0.159556 0.160294 0.154109 0.157851 0.154439 0.158792 0.152938 0.154898 0.155076 0.153294 0.151487 0.155916 0.155127 0.159225 0.155941 0.158360 0.152530 0.152785 0.164444 0.165258 0.175160 0.163705 0.166022 0.166200 0.163782 H_oc_A311_jigu 2 0.189568 0.051115 0.000000 0.164011 0.167065 0.164138 0.163629 0.163171 0.163298 0.162051 0.166735 0.161007 0.162509 0.164673 0.159709 0.152479 0.153549 0.157112 0.155152 0.152581 0.159251 0.152225 0.156298 0.155840 0.153421 0.156476 0.159963 0.155229 0.154669 0.149654 0.158665 0.152047 0.157443 0.152581 0.152301 0.158945 0.159072 0.154465 0.156476 0.153574 0.158436 0.151512 0.154490 0.155636 0.151359 0.150112 0.156629 0.155280 0.158767 0.154159 0.158614 0.151563 0.153650 0.163833 0.163018 0.173633 0.161974 0.164647 0.164164 0.161796 H_oc_T15847_jigu 3 0.190968 0.164214 0.164011 0.000000 0.055442 0.053584 0.059032 0.056079 0.058395 0.058777 0.054755 0.061195 0.056792 0.054119 0.059032 0.134329 0.133413 0.129900 0.135424 0.134839 0.131835 0.132904 0.128831 0.137639 0.135984 0.133184 0.130562 0.135144 0.133209 0.131555 0.131860 0.134202 0.135373 0.130358 0.129111 0.134431 0.130638 0.134075 0.129569 0.134202 0.134228 0.132344 0.133286 0.126336 0.135959 0.134966 0.134762 0.135959 0.135780 0.133820 0.132828 0.136060 0.136010 0.136468 0.134533 0.135373 0.131911 0.127151 0.125751 0.129391 H_oc_T355_jigu 4 0.193158 0.166862 0.167065 0.055442 0.000000 0.053533 0.058421 0.057148 0.058395 0.059083 0.055111 0.061246 0.062697 0.055086 0.063003 0.134991 0.133617 0.131529 0.137053 0.136773 0.134228 0.135959 0.131326 0.140133 0.137664 0.136239 0.132904 0.137079 0.136671 0.133540 0.133540 0.135679 0.138733 0.134177 0.130689 0.135246 0.133744 0.136213 0.131606 0.137206 0.134788 0.134635 0.135068 0.129900 0.136875 0.135628 0.136035 0.138250 0.136850 0.137588 0.133897 0.138453 0.136773 0.138657 0.134991 0.136290 0.132217 0.128118 0.125242 0.130308 H_ro_A547_ma 5 0.190485 0.163069 0.164138 0.053584 0.053533 0.000000 0.057072 0.057479 0.057352 0.056002 0.055086 0.060050 0.058192 0.055061 0.059821 0.132879 0.131606 0.128144 0.132548 0.133082 0.131046 0.132268 0.129366 0.136442 0.133769 0.133209 0.129569 0.133795 0.131606 0.130002 0.130715 0.133311 0.134635 0.129518 0.128169 0.131860 0.129238 0.132675 0.129086 0.133159 0.131606 0.131758 0.130766 0.125751 0.133286 0.133668 0.133362 0.134660 0.134279 0.133693 0.131937 0.136086 0.134100 0.134915 0.133184 0.132904 0.129697 0.125751 0.123078 0.128144 H_ro_J213_ma 6 0.189772 0.163273 0.163629 0.059032 0.058421 0.057072 0.000000 0.052337 0.054195 0.051268 0.055646 0.056130 0.056512 0.

top=structure assignments, bottom=a priori assignments and you can see they are nearly identical